In [1]:
%%capture
! pip install pytorch-lightning
! pip install pytorch-lightning-bolts

In [2]:
!pip install torchvision

In [3]:
import os

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
from torchmetrics.functional import accuracy
from pl_bolts.datasets import DummyDataset

/Users/daohuei/miniconda3/envs/py3_8/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/daohuei/miniconda3/envs/py3_8/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowExxb
  Referenced from: /Users/daohuei/miniconda3/envs/py3_8/lib/python3.8/site-packages/torchvision/image.so
  Expected in: /Users/daohuei/miniconda3/envs/py3_8/lib/python3.8/site-packages/torch/lib/libc10.dylib
  warn(f"Failed to load image Python extension: {e}")
/Users/daohuei/miniconda3/envs/py3_8/lib/python3.8/site-packages/pl_bolts/utils/warnings.py:30: UserWarning: You want to use `wandb` which is not installed yet, install it with `pip install wandb`.
  stdout_func(
/Users/daohuei/miniconda3/envs/py3_8/lib/python3.8/site-packages/pl_bolts/utils/warnings.py:30: UserWarning: You want to use `gym` which is not installed yet, install it with `pip install gym`.
  stdout_func(


In [4]:
train = DummyDataset((1, 28, 28), (1,))
train = DataLoader(train, batch_size=32)

In [5]:
val = DummyDataset((1, 28, 28), (1,))
val = DataLoader(val, batch_size=32)

In [6]:
test = DummyDataset((1, 28, 28), (1,))
test = DataLoader(test, batch_size=32)

In [14]:
class LitAutoEncoder(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(28 * 28, 128), nn.ReLU(), nn.Linear(128, 3))
        self.decoder = nn.Sequential(nn.Linear(3, 128), nn.ReLU(), nn.Linear(128, 28 * 28))

    def training_step(self, batch, batch_idx, optimizer_idx=None):
        # --------------------------
        # REPLACE WITH YOUR OWN
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        self.log('train_loss', loss)
        return [loss, loss]
        # --------------------------

    def validation_step(self, batch, batch_idx):
        # --------------------------
        # REPLACE WITH YOUR OWN
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        self.log('val_loss', loss)
        # --------------------------

    def test_step(self, batch, batch_idx):
        # --------------------------
        # REPLACE WITH YOUR OWN
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        self.log('test_loss', loss)
        # --------------------------

    def configure_optimizers(self):
        optimizer_1 = torch.optim.Adam(self.parameters(), lr=1e-3)
        optimizer_2 = torch.optim.Adam(self.parameters(), lr=1e-3)
        return [optimizer_1, optimizer_2]

In [ ]:
# init model
ae = LitAutoEncoder()

# Initialize a trainer
trainer = pl.Trainer(gpus=0, max_epochs=3, progress_bar_refresh_rate=20)

# Train the model ⚡
trainer.fit(ae, train, val)

In [ ]:
trainer.test(dataloaders=test)

/Users/daohuei/miniconda3/envs/py3_8/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1444: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  rank_zero_warn(
Restoring states from the checkpoint path at /Users/daohuei/CodeNest/code-style-probing/lightning_logs/version_2/checkpoints/epoch=2-step=939.ckpt
Loaded model weights from checkpoint at /Users/daohuei/CodeNest/code-style-probing/lightning_logs/version_2/checkpoints/epoch=2-step=939.ckpt
/Users/daohuei/miniconda3/envs/py3_8/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` ar

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.08343713730573654
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.08343713730573654}]

In [30]:
# Start tensorboard.
%load_ext tensorboard
%tensorboard --logdir lightning_logs/